# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"id_persona"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Se considera como **vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

### Cargar Librerías

In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

### Diccionarios para codificar variables

In [2]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

### Dataset de casos positivos

In [3]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})
# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas = df_cas.dropna()
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32', 'id_persona': 'int32'})

df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['id_persona']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('int8')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8') 

In [4]:
df_cas

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento
0,7,20200720,1,1,54,4,16
1,11,20200701,2,2,56,4,25
2,12,20210413,3,2,62,5,25
3,30,20210628,2,2,55,4,25
4,37,20210429,3,1,74,6,25
...,...,...,...,...,...,...,...
2513972,37293491,20201004,2,1,41,3,15
2513973,37293520,20200516,1,2,34,2,15
2513974,37293584,20200513,2,1,80,7,15
2513975,37293776,20210114,2,2,81,7,14


### Dataset de fallecidos

In [5]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'id_persona']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})
df_fal = df_fal.dropna()  # Drop nan values
df_fal['fallecido'] = 1
df_fal = df_fal.astype({'id_persona': 'int32', 'fallecido': 'int8'})

# Transformar a formato fecha
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento'}, inplace=True)
df_fal = df_fal[['id_persona', 'fallecido', 'fecha_fallecimiento']]
df_fal

,id_persona,fallecido,fecha_fallecimiento
0,24773171,1,2021-04-09
1,24702378,1,2021-05-01
2,24737410,1,2021-06-02
3,24737438,1,2021-04-10
4,24737528,1,2021-04-23
...,...,...,...
200032,19196702,1,2021-05-03
200033,19008930,1,2021-06-20
200034,19009513,1,2021-05-01
200035,19030792,1,2021-05-30


### Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [6]:
df = df_cas.merge(df_fal, on = 'id_persona', how = 'left')
df['fallecido'].fillna(0, inplace=True)
df['fallecido'] = df['fallecido'].astype('int8')
del df_fal, df_cas

# IMPORTANTE, se repiten pruebas en algunos
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento
0,7,20200720,1,1,54,4,16,0,NaT
1,11,20200701,2,2,56,4,25,0,NaT
2,12,20210413,3,2,62,5,25,1,2021-04-20
3,30,20210628,2,2,55,4,25,0,NaT
4,37,20210429,3,1,74,6,25,1,2021-05-08
...,...,...,...,...,...,...,...,...,...
2514006,37293491,20201004,2,1,41,3,15,0,NaT
2514007,37293520,20200516,1,2,34,2,15,0,NaT
2514008,37293584,20200513,2,1,80,7,15,1,2020-05-27
2514009,37293776,20210114,2,2,81,7,14,1,2021-01-10


### Convertir en dask dataframe (ddf) para poder trabajar con el ddf de vacunados

In [7]:
df = dd.from_pandas(df, npartitions=3)

### Dataset de vacunados

In [8]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int64'})

# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
""" ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce")) """

ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,505864,2,20211209,5
1,7676322,2,20211111,6
2,7752439,1,20211216,5
3,7638125,1,20210913,6
4,7507035,2,20211008,6


### Separar información de dosis recibidas para juntarlos con nuestro dataframe

In [9]:
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')
del df_dosis_3, ddf_vac

### Convertir nuestro ddf en un pandas dataframe

In [10]:
df = df.compute()

### Rellenar espacios vacíos con 0

In [11]:
# Variable "vacunado" si recibe las 2 dosis
df['vacunado'] = np.where(df['dosis_2'] == 1, 1, 0).astype('int8')

df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

### Ordenar y Guardar el df

In [12]:
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento', 
         'fallecido', 'fecha_fallecimiento', 
         'vacunado', 'dosis_1', 'dosis_2', 'dosis_3', 
         'id_vacuna_1', 'id_vacuna_2', 'id_vacuna_3' ,'fecha_vacuna_1','fecha_vacuna_2', 'fecha_vacuna_3']]

In [13]:
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
0,488,20210404,3,2,72,6,25,0,NaT,1,1,1,0,1,2,0,20210426.0,20210609.0,NaN
1,879,20200811,1,2,54,4,25,0,NaT,0,0,0,0,0,0,0,NaN,NaN,NaN
2,889,20200418,1,2,84,7,25,0,NaT,1,1,1,1,1,2,5,20210422.0,20210609.0,20211213.0
3,1881,20201003,1,2,67,5,15,0,NaT,1,1,1,1,5,5,5,20210522.0,20210612.0,20211130.0
4,1989,20200619,1,1,69,5,25,0,NaT,1,1,1,1,1,2,5,20210428.0,20210609.0,20211209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64495,37277325,20200723,1,1,39,2,15,0,NaT,1,1,1,0,6,6,0,20210922.0,20211013.0,NaN
64496,37279726,20200521,2,2,57,4,14,1,2020-05-21,0,0,0,0,0,0,0,NaN,NaN,NaN
64497,37285740,20200401,2,1,34,2,7,0,NaT,1,1,1,0,6,6,0,20211009.0,20211101.0,NaN
64498,37286289,20210603,3,2,49,3,13,0,NaT,0,0,0,0,0,0,0,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2514178 entries, 0 to 64499
Data columns (total 19 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id_persona           int32         
 1   fecha_resultado      int32         
 2   metododx             int8          
 3   sexo                 int8          
 4   edad                 int8          
 5   edad_cat             int8          
 6   departamento         int8          
 7   fallecido            int8          
 8   fecha_fallecimiento  datetime64[ns]
 9   vacunado             int8          
 10  dosis_1              int8          
 11  dosis_2              int8          
 12  dosis_3              int8          
 13  id_vacuna_1          int8          
 14  id_vacuna_2          int8          
 15  id_vacuna_3          int8          
 16  fecha_vacuna_1       float64       
 17  fecha_vacuna_2       float64       
 18  fecha_vacuna_3       float64       
dtypes: datetime64[ns](1), f

In [15]:
df.to_csv('../Data/DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv',  index=False)